## Set working directory

https://aeturrell.github.io/coding-for-economists/text-intro.html

In [1]:
import pathlib   # Change cwd
import os 

path = pathlib.Path.cwd().parent
os.chdir(path)

## Imports

In [2]:
import yaml   # 3rd party packages
import warnings
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport

from digex_src import config    # Local imports
from digex_src import preprocess_data
from digex_src.load_data import get_data_filepath

warnings.filterwarnings('ignore')    # Ignore warnings